In [ ]:
from transformer_lens import HookedTransformer
import torch
import matplotlib.pyplot as plt
import os
from einops import einsum


torch.set_grad_enabled(False)

model = HookedTransformer.from_pretrained("meta-llama/Meta-Llama-3-8B", device="cuda")

In [ ]:
A = """
(28, 59)
(86, 175)
(13, 29)
(37, 77)
(55, 113)
(84, 171)
(66, 135)
(85, 173)
(27, 57)
(15, 33)
(94, 191)
(37, 77)
(14, 31)
(42, 87)
"""



In [ ]:
W_U = model.unembed.W_U

tokens_ids = model.to_tokens(A)
tokens = model.tokenizer.convert_ids_to_tokens(tokens_ids)
space_token_indexes = torch.tensor(
    [i for i, token in enumerate(tokens) if token == "Ġ"] # weird gpt2 stuffs
)
# these are the tokens before a given prediction

logits, activation_cache = model.run_with_cache(A)

In [ ]:
accum_resids, labels = activation_cache.accumulated_resid(return_labels=True, apply_ln=True) # layer, batch, pos, dim
accum_resids = einsum("l b p d -> l p d", accum_resids) # get rid of batch dim
accum_resids = einsum("l p d, d v -> l p v", accum_resids, W_U)
accum_resids = accum_resids[:, space_token_indexes, :]

for layer in accum_resids:
    # pos, v
    # find top k at each pos
    top_k = torch.topk(layer, 1, dim=1)
    # reshape to a 1d tensor
    top_k = top_k.indices.flatten()
    print(model.tokenizer.convert_ids_to_tokens(top_k))